In [ ]:
# Configuración de cores seguros para datasets grandes
import os
N_JOBS = 16  # Reducido drásticamente para evitar memory overflow
os.environ['OMP_NUM_THREADS'] = str(N_JOBS)
os.environ['MKL_NUM_THREADS'] = str(N_JOBS)
os.environ['OPENBLAS_NUM_THREADS'] = str(N_JOBS)
print(f"Usando {N_JOBS} cores (configuración segura para datasets grandes)")

Usando 16 cores (configuración segura para datasets grandes)


In [ ]:
!pip install scikit-learn-intelex xgboost
from sklearnex import patch_sklearn
patch_sklearn()
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession

Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time


In [ ]:
df_mod_clean = pd.read_parquet('obt_trips_2022_2025_clean.parquet')

In [ ]:
df_mod_clean

,pickup_hour,pickup_dow,pickup_month,pickup_year,pickup_borough,vendor_name,rate_code_desc,passenger_count,total_amount,service_type,is_weekend,is_rush_hour
0,0.0,0.0,1.0,2022.0,Manhattan,2,1.0,2.0,20.80,yellow,1,0
1,0.0,0.0,1.0,2022.0,Manhattan,1,1.0,2.0,8.30,yellow,1,0
2,0.0,0.0,1.0,2022.0,Manhattan,2,1.0,1.0,12.25,yellow,1,0
3,0.0,0.0,1.0,2022.0,Manhattan,1,1.0,1.0,19.55,yellow,1,0
4,0.0,0.0,1.0,2022.0,Manhattan,1,1.0,1.0,24.30,yellow,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1836494,23.0,6.0,8.0,2025.0,Brooklyn,2,1.0,1.0,16.68,green,1,0
1836495,23.0,6.0,8.0,2025.0,Queens,2,1.0,1.0,5.50,green,1,0
1836496,23.0,6.0,8.0,2025.0,Queens,2,1.0,2.0,17.52,green,1,0
1836497,23.0,6.0,8.0,2025.0,Queens,2,5.0,4.0,51.25,green,1,0


In [ ]:
df_train = df_mod_clean[df_mod_clean["pickup_year"].isin([2022, 2023])]
df_val = df_mod_clean[df_mod_clean["pickup_year"] == 2024]
df_test = df_mod_clean[df_mod_clean["pickup_year"] == 2025]
print(f"Train shape: {df_train.shape}, Val shape: {df_val.shape}, Test shape: {df_test.shape}")

target = "total_amount"
feature_cols = [col for col in df_mod_clean.columns if col != target]

X_train = df_train.drop(columns=[target])
y_train = df_train[target]

X_val = df_val.drop(columns=[target])
y_val = df_val[target]

X_test = df_test.drop(columns=[target])
y_test = df_test[target]


Train shape: (1013783, 12), Val shape: (483652, 12), Test shape: (339064, 12)


In [ ]:
df_mod_clean.columns

Index(['pickup_hour', 'pickup_dow', 'pickup_month', 'pickup_year',
       'pickup_borough', 'vendor_name', 'rate_code_desc', 'passenger_count',
       'total_amount', 'service_type', 'is_weekend', 'is_rush_hour'],
      dtype='object')

In [ ]:
#Procesamiento
categorical = ['vendor_name', 'rate_code_desc','service_type','pickup_borough','is_weekend','is_rush_hour']
numeric_features = ['passenger_count','pickup_year'] 

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical)
])

# Escalar usando solo datos de entrenamiento
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()

X_train_scaled = preprocessor.fit_transform(X_train)
X_val_scaled = preprocessor.transform(X_val)
X_test_scaled = preprocessor.transform(X_test)


In [ ]:
preprocessor_hot = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical)
])

X_train_hot = preprocessor_hot.fit_transform(X_train)
X_val_hot= preprocessor_hot.transform(X_val)
X_test_hot = preprocessor_hot.transform(X_test)


In [ ]:
#Baseline 

  # import numpy as np
  # from sklearn.metrics import mean_squared_error, mean_absolute_error

  # # 1. Calcular media del target en entrenamiento
  # y_train_mean = y_train.mean()

  # # 2. Predecir la media para validación y test
  # y_val_pred_baseline = np.full(len(y_val), y_train_mean)
  # y_test_pred_baseline = np.full(len(y_test), y_train_mean)

  # # 3. Calcular métricas
  # # Validación
  # val_rmse_baseline = np.sqrt(mean_squared_error(y_val, y_val_pred_baseline))
  # val_mae_baseline = mean_absolute_error(y_val, y_val_pred_baseline)

  # # Test  
  # test_rmse_baseline = np.sqrt(mean_squared_error(y_test, y_test_pred_baseline))
  # test_mae_baseline = mean_absolute_error(y_test, y_test_pred_baseline)

  # print(f"Baseline (Media):")
  # print(f"Validación - RMSE: {val_rmse_baseline:.4f}, MAE: {val_mae_baseline:.4f}")
  # print(f"Test - RMSE: {test_rmse_baseline:.4f}, MAE: {test_mae_baseline:.4f}")

  # ¿Por qué es útil?

  # El baseline te da un punto de referencia mínimo. Si tu modelo complejo no supera este baseline,
  # algo está muy mal.

  # Ejemplo interpretación:

  # - Baseline RMSE: 15.2
  # - Tu modelo RMSE: 8.7
  # - Mejora: 43% mejor que predecir siempre la media

  # Baseline alternativo (Regresión Lineal simple):

  # from sklearn.linear_model import LinearRegression

  # # Solo con una feature importante
  # lr_baseline = LinearRegression()
  # lr_baseline.fit(X_train[['trip_distance']], y_train)

  # y_val_pred_lr = lr_baseline.predict(X_val[['trip_distance']])
  # val_rmse_lr = np.sqrt(mean_squared_error(y_val, y_val_pred_lr))



In [ ]:
  # Combinar train + val
X_combined = np.vstack([X_train_scaled, X_val_scaled])
y_combined = np.hstack([y_train, y_val])


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# 1. Definir 3 regresores base diferentes
estimators = [
    ('linear', LinearRegression()),
    ('forest', RandomForestRegressor(random_state=42)),
    ('ridge', Ridge())
]

# 2. Hard Voting (promedio simple)
voting_hard = VotingRegressor(
    estimators=estimators,
    n_jobs=N_JOBS
  )


# Hiperparámetros para cada modelo base
param_grid_voting = {

    # RandomForest parameters (prefijo: forest__)
    'forest__n_estimators': [50],
    'forest__max_depth': [10],
    'forest__min_samples_split': [2],

    # SVR parameters (prefijo: svm__)
    'ridge__alpha': [0.1, 1.0, 10.0],
  }

train_idx = np.arange(len(X_train))                     
val_idx = np.arange(len(X_train), len(X_combined))

  # GridSearch
grid_voting = GridSearchCV(
    estimator=voting_hard,
    param_grid=param_grid_voting,
    cv = [(train_idx, val_idx)],  #Split no la entendi jaja
    scoring='neg_mean_squared_error',
    n_jobs=N_JOBS,
    verbose=1
)

In [ ]:
pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
print(torch.backends.mps.is_available())


False


In [ ]:
  # Entrenar
grid_voting.fit(X_combined, y_combined)

Fitting 1 folds for each of 3 candidates, totalling 3 fits


/home/dchicaiza/Pset-5/.venv/lib/python3.10/site-packages/daal4py/sklearn/_n_jobs_support.py:83: UserWarning: 'Threading' parallel backend is not supported by Extension for Scikit-learn*. Falling back to usage of all available threads.
  warn(
/home/dchicaiza/Pset-5/.venv/lib/python3.10/site-packages/daal4py/sklearn/_n_jobs_support.py:83: UserWarning: 'Threading' parallel backend is not supported by Extension for Scikit-learn*. Falling back to usage of all available threads.
  warn(
/home/dchicaiza/Pset-5/.venv/lib/python3.10/site-packages/daal4py/sklearn/_n_jobs_support.py:83: UserWarning: 'Threading' parallel backend is not supported by Extension for Scikit-learn*. Falling back to usage of all available threads.
  warn(
/home/dchicaiza/Pset-5/.venv/lib/python3.10/site-packages/daal4py/sklearn/_n_jobs_support.py:83: UserWarning: 'Threading' parallel backend is not supported by Extension for Scikit-learn*. Falling back to usage of all available threads.
  warn(
/home/dchicaiza/Pset-5/.

,estimator,VotingRegress... n_jobs=16)
,param_grid,"{'forest__max_depth': [10], 'forest__min_samples_split': [2], 'forest__n_estimators': [50], 'ridge__alpha': [0.1, 1.0, ...]}"
,scoring,'neg_mean_squared_error'
,n_jobs,16
,refit,True
,cv,"[(array([ ...pe=(1013783,)), ...)]"
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,fit_intercept,True


In [ ]:
  # Mejores parámetros
print("Mejores parámetros:", grid_voting.best_params_)

  # Predicciones
y_val_pred_voting = grid_voting.predict(X_val_scaled)
y_test_pred_voting = grid_voting.predict(X_test_scaled)

  # Métricas
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred_voting))
val_mae = mean_absolute_error(y_val, y_val_pred_voting)
val_r2 = r2_score(y_val, y_val_pred_voting)

print(f"Voting - Validación RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}")

Mejores parámetros: {'forest__max_depth': 10, 'forest__min_samples_split': 2, 'forest__n_estimators': 50, 'ridge__alpha': 10.0}
Voting - Validación RMSE: 9.8716, MAE: 7.6808


In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

base_tree = DecisionTreeRegressor(random_state=42)

  # BAGGING 
bagging_model = BaggingRegressor(
    estimator=base_tree,
    n_estimators=100,
    bootstrap=True,        
    random_state=42,
    n_jobs=4  # Limitado para bagging interno
  )

# PASTING  
pasting_model = BaggingRegressor(
    estimator=base_tree,
    n_estimators=100,
    bootstrap=False,       
    random_state=42,
    n_jobs=4  # Limitado para pasting interno
)



  # Parámetros REDUCIDOS para evitar memory overflow
param_grid_bagging = {
    'n_estimators': [50, 100],                    # Reducido de 3 a 2 opciones
    'max_samples': [0.7, 1.0],                    # Reducido de 3 a 2 opciones
    'max_features': [0.7, 1.0],                   # Reducido de 3 a 2 opciones
    'estimator__max_depth': [5, 10, None],        # Reducido de 4 a 3 opciones
    'estimator__min_samples_split': [2, 10]       # Reducido de 3 a 2 opciones
}
# Total: 2*2*2*3*2 = 48 combinaciones (vs 324 anteriores)

grid_bagging = GridSearchCV(
    estimator=bagging_model,
    param_grid=param_grid_bagging,
    cv=[(train_idx, val_idx)],
    scoring='neg_mean_squared_error',
    n_jobs=N_JOBS,  # Paralelización en GridSearch
    verbose=2,      # Más información para debugging
    pre_dispatch='2*n_jobs'  # Control de memoria
)


grid_pasting = GridSearchCV(
    estimator=pasting_model,
    param_grid=param_grid_bagging, 
    cv=[(train_idx, val_idx)],
    scoring='neg_mean_squared_error',
    n_jobs=N_JOBS,
    verbose=2,
    pre_dispatch='2*n_jobs'
)

In [ ]:
grid_bagging.fit(X_combined, y_combined)

Fitting 1 folds for each of 48 candidates, totalling 48 fits
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=1.0, n_estimators=50; total time=  14.8s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=1.0, n_estimators=50; total time=  14.8s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=0.7, n_estimators=50; total time=  16.0s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=0.7, n_estimators=50; total time=  16.0s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=50; total time=  17.3s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=50; total time=  17.3s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=1.0, max_samples=0.7, n_estimators=50; t

/home/dchicaiza/Pset-5/.venv/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END estimator__max_depth=10, estimator__min_samples_split=10, max_features=0.7, max_samples=1.0, n_estimators=100; total time=  34.7s
[CV] END estimator__max_depth=None, estimator__min_samples_split=2, max_features=0.7, max_samples=1.0, n_estimators=50; total time=  19.0s
[CV] END estimator__max_depth=None, estimator__min_samples_split=2, max_features=0.7, max_samples=1.0, n_estimators=50; total time=  19.0s
[CV] END estimator__max_depth=10, estimator__min_samples_split=10, max_features=1.0, max_samples=1.0, n_estimators=50; total time=  33.4s
[CV] END estimator__max_depth=10, estimator__min_samples_split=10, max_features=1.0, max_samples=1.0, n_estimators=50; total time=  33.4s
[CV] END estimator__max_depth=None, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=100; total time=  29.1s
[CV] END estimator__max_depth=None, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=100; total time=  29.1s
[CV] END estimator__max_dep

,estimator,BaggingRegres...ndom_state=42)
,param_grid,"{'estimator__max_depth': [5, 10, ...], 'estimator__min_samples_split': [2, 10], 'max_features': [0.7, 1.0], 'max_samples': [0.7, 1.0], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,16
,refit,True
,cv,"[(array([ ...pe=(1013783,)), ...)]"
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [ ]:
grid_pasting.fit(X_combined, y_combined)

Fitting 1 folds for each of 48 candidates, totalling 48 fits
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=50; total time=  22.1s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=50; total time=  22.1s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=0.7, n_estimators=50; total time=  23.7s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=0.7, n_estimators=50; total time=  23.7s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=1.0, n_estimators=50; total time=  24.7s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=1.0, n_estimators=50; total time=  24.8s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=1.0, n_estimators=50; to

,estimator,BaggingRegres...ndom_state=42)
,param_grid,"{'estimator__max_depth': [5, 10, ...], 'estimator__min_samples_split': [2, 10], 'max_features': [0.7, 1.0], 'max_samples': [0.7, 1.0], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,16
,refit,True
,cv,"[(array([ ...pe=(1013783,)), ...)]"
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [ ]:
# Para CatBoost necesitamos datos SIN codificar con categorías como strings
X_train_catboost = X_train.copy()
X_val_catboost = X_val.copy()

# Convertir columnas categóricas a string
for col in categorical:
    X_train_catboost[col] = X_train_catboost[col].astype(str)
    X_val_catboost[col] = X_val_catboost[col].astype(str)

X_combined_catboost = pd.concat([X_train_catboost, X_val_catboost], axis=0, ignore_index=True)

# Obtener índices de columnas categóricas
cat_indices = [X_train_catboost.columns.get_loc(col) for col in categorical]
print(f"Índices de columnas categóricas: {cat_indices}")
print(f"Shape X_combined_catboost: {X_combined_catboost.shape}")
print(f"Dtypes de categorías: {X_combined_catboost[categorical].dtypes.to_dict()}")

Índices de columnas categóricas: [5, 6, 8, 4, 9, 10]
Shape X_combined_catboost: (1497435, 11)
Dtypes de categorías: {'vendor_name': dtype('O'), 'rate_code_desc': dtype('O'), 'service_type': dtype('O'), 'pickup_borough': dtype('O'), 'is_weekend': dtype('O'), 'is_rush_hour': dtype('O')}


In [ ]:
#CatBoost

from catboost import CatBoostRegressor

catboost_model = CatBoostRegressor(
    random_seed=42,
    verbose=False,
    task_type='GPU',
    devices='0',
    cat_features=cat_indices,  # USAR ÍNDICES en lugar de nombres
    thread_count=N_JOBS  # Controlar threads en GPU
)

  # Parámetros REDUCIDOS para CatBoost
param_grid_catboost = {
    'iterations': [100, 300],              # Reducido de 3 a 2 opciones
    'learning_rate': [0.01, 0.1],          # Reducido de 3 a 2 opciones
    'depth': [4, 6, 8],                    # Mantener 3 opciones
    'l2_leaf_reg': [1, 3],                 # Reducido de 3 a 2 opciones
}
# Total: 2*2*3*2 = 24 combinaciones (vs 243 anteriores)

grid_catboost = GridSearchCV(
    estimator=catboost_model,
    param_grid=param_grid_catboost,
    cv=[(np.arange(len(X_train)), np.arange(len(X_val)))],
    scoring='neg_mean_squared_error',
    n_jobs=1,  # CatBoost ya usa paralelización interna con GPU
    verbose=2
)

In [ ]:
# Usar datos SIN codificar para CatBoost
grid_catboost.fit(X_combined_catboost, y_combined)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   3.1s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   3.1s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.1; total time=   3.0s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.1; total time=   3.0s
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.01; total time=   3.0s
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.01; total time=   3.0s
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.1; total time=   3.1s
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.1; total time=   3.1s
[CV] END depth=4, iterations=300, l2_leaf_reg=1, learning_rate=0.01; total time=   5.0s
[CV] END depth=4, iterations=300, l2_leaf_reg=1, learning_rate=0.01; total time=   5.0s
[CV] END depth=4, iterations=300, l2_leaf_reg=1, learning_rate=

,estimator,<catboost.cor...x7f8fdd8ff460>
,param_grid,"{'depth': [4, 6, ...], 'iterations': [100, 300], 'l2_leaf_reg': [1, 3], 'learning_rate': [0.01, 0.1]}"
,scoring,'neg_mean_squared_error'
,n_jobs,1
,refit,True
,cv,"[(array([ ...pe=(1013783,)), ...)]"
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False


In [ ]:
# Resumen completo de todos los modelos entrenados

print("="*80)
print("RESUMEN DE RESULTADOS - MODELOS DE PREDICCIÓN DE TOTAL_AMOUNT")
print("="*80)
print(f"\nDATASET:")
print(f"  - Train: {df_train.shape[0]:,} muestras")
print(f"  - Validación: {df_val.shape[0]:,} muestras")
print(f"  - Test: {df_test.shape[0]:,} muestras")
print(f"  - Features: {len(feature_cols)}")
print(f"  - Cores utilizados: {N_JOBS}")

print("\n" + "="*80)
print("1. VOTING REGRESSOR")
print("="*80)
print(f"Mejores parámetros: {grid_voting.best_params_}")
print(f"Mejor score (neg MSE): {grid_voting.best_score_:.4f}")
print(f"\nMétricas en Validación:")
print(f"  - RMSE: {val_rmse:.4f}")
print(f"  - MAE:  {val_mae:.4f}")
print(f"  - R²:   {val_r2:.4f}")

print("\n" + "="*80)
print("2. BAGGING REGRESSOR")
print("="*80)
print(f"Mejores parámetros: {grid_bagging.best_params_}")
print(f"Mejor score (neg MSE): {grid_bagging.best_score_:.4f}")
bagging_rmse = np.sqrt(-grid_bagging.best_score_)
print(f"RMSE estimado: {bagging_rmse:.4f}")

print("\n" + "="*80)
print("3. PASTING REGRESSOR")
print("="*80)
print(f"Mejores parámetros: {grid_pasting.best_params_}")
print(f"Mejor score (neg MSE): {grid_pasting.best_score_:.4f}")
pasting_rmse = np.sqrt(-grid_pasting.best_score_)
print(f"RMSE estimado: {pasting_rmse:.4f}")

print("\n" + "="*80)
print("4. CATBOOST REGRESSOR")
print("="*80)
print(f"Mejores parámetros: {grid_catboost.best_params_}")
print(f"Mejor score (neg MSE): {grid_catboost.best_score_:.4f}")
catboost_rmse = np.sqrt(-grid_catboost.best_score_)
print(f"RMSE estimado: {catboost_rmse:.4f}")

print("\n" + "="*80)
print("COMPARACIÓN FINAL")
print("="*80)
results_summary = pd.DataFrame({
    'Modelo': ['Voting', 'Bagging', 'Pasting', 'CatBoost'],
    'RMSE': [val_rmse, bagging_rmse, pasting_rmse, catboost_rmse],
    'Neg MSE Score': [grid_voting.best_score_, grid_bagging.best_score_, 
                      grid_pasting.best_score_, grid_catboost.best_score_]
})
results_summary = results_summary.sort_values('RMSE')
print(results_summary.to_string(index=False))

print("\n" + "="*80)
best_model = results_summary.iloc[0]['Modelo']
best_rmse = results_summary.iloc[0]['RMSE']
print(f"🏆 MEJOR MODELO: {best_model} con RMSE = {best_rmse:.4f}")
print("="*80)

RESUMEN DE RESULTADOS - MODELOS DE PREDICCIÓN DE TOTAL_AMOUNT

DATASET:
  - Train: 1,013,783 muestras
  - Validación: 483,652 muestras
  - Test: 339,064 muestras
  - Features: 11
  - Cores utilizados: 16

1. VOTING REGRESSOR
Mejores parámetros: {'forest__max_depth': 10, 'forest__min_samples_split': 2, 'forest__n_estimators': 50, 'ridge__alpha': 10.0}
Mejor score (neg MSE): -107.2164

Métricas en Validación:
  - RMSE: 9.8716
  - MAE:  7.6808
  - R²:   0.0639

2. BAGGING REGRESSOR
Mejores parámetros: {'estimator__max_depth': 10, 'estimator__min_samples_split': 10, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 100}
Mejor score (neg MSE): -95.1661
RMSE estimado: 9.7553

3. PASTING REGRESSOR
Mejores parámetros: {'estimator__max_depth': 10, 'estimator__min_samples_split': 10, 'max_features': 1.0, 'max_samples': 0.7, 'n_estimators': 50}
Mejor score (neg MSE): -95.1712
RMSE estimado: 9.7556

4. CATBOOST REGRESSOR
Mejores parámetros: {'depth': 8, 'iterations': 300, 'l2_leaf_reg': 1, 